In [88]:
import pretty_midi
import numpy as np
import pickle

num_notes = 128 # Number of pitches in MIDI
num_time_shifts = 100
min_time_shift = 0.01

In [167]:
data = np.load('baseline_vae_midis/recording_vae.npy', allow_pickle=True)

In [168]:
def data_to_midi(data, instrument_number):
    instrument=pretty_midi.Instrument(program=instrument_number,name=pretty_midi.program_to_instrument_name(instrument_number))
    curr_time = 0
    #start time of the current note being played
    note_start = {}
    note_same = {}
    for i, idx in enumerate(data):
        if idx < num_notes:
            #note on event
            if idx in note_start:
                note_same[idx] += 1
                continue
            note_start[idx] = curr_time
            note_same[idx] = 1
        elif idx < 2*num_notes:
            #note off event
            curr_note = idx - num_notes
            if curr_note not in note_start:
                continue
                #raise ValueError("note" + str(idx-num_notes) + "has not been started!")
            note_same[curr_note] -= 1
            if note_same[curr_note] == 0:
                note=pretty_midi.Note(velocity=96, pitch=curr_note, start=note_start[curr_note], end=curr_time)
                instrument.notes.append(note)
                del note_start[curr_note]
                del note_same[curr_note]
        elif idx < 2*num_notes + num_time_shifts:
            #time shift event
            curr_time += (idx - 2*num_notes + 1)*min_time_shift
    return instrument
    

In [169]:
recording = pretty_midi.PrettyMIDI()
recording.instruments.append(data_to_midi(data,0))

In [170]:
recording.write("baseline_vae_midis/sample.mid")